In [27]:

from hyperopt import fmin, tpe, hp, Trials
import numpy as np
import pandas as pd
import tensorflow as tf
import csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error


In [28]:
df = pd.read_csv('exchange_rate_with_missing.csv')
#fill missing values (does not work when the first values are missing)
df.interpolate(method="linear", direction = "forward", inplace= True)
#change date type from object to date
df['date'] = pd.to_datetime(df['date'])
#set date as index
df.set_index('date', inplace=True)


/tmp/ipykernel_3112/3721340139.py:3: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df.interpolate(method="linear", direction = "forward", inplace= True)


In [4]:
#For LSTM data should be normalized but in our example it is already in the range of 0 adn 1
for x in df['6'] : 
    if x<0 and x>1 :
        print(False)
 

In [29]:
#add feature '6+1' for t+1 values
df['6+1'] = df['6'].shift(-1) 

#separate train and test datasets 
index = int(len(df) * 0.8)
train_data_x = df['6'].iloc[:index].values
train_data_y = df['6+1'].iloc[:index].values
test_data_x = df['6'].iloc[index:].values
test_data_y = df['6+1'].iloc[index:].values

#print(test_data_x.shape)

#The LSTM network expects the input data (X) to be provided with a specific array structure in the form of [samples, time steps, features]
train_data_x = train_data_x.reshape(train_data_x.shape[0], 1, 1)
test_data_x = test_data_x.reshape(test_data_x.shape[0], 1, 1)

def lstm_model(params):
    model = Sequential()
    model.add(LSTM(params['units'], input_shape=(1, 1)))
    model.add(Dense(1))
    optimizer = Adam(learning_rate=params['learning_rate'])  # Correctly specifying the learning rate here
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    model.fit(train_data_x, train_data_y, epochs=params['epochs'], batch_size=params['batch_size'], verbose=0)
    # Evaluate the model on validation data and return the loss
    val_loss = model.evaluate(train_data_x, train_data_y)
    return val_loss

# Define the search space for hyperparameters
space = {
    'units': hp.choice('units', [50, 100, 150]),
    'learning_rate': hp.uniform('learning_rate', 0.0001, 0.1),
    'epochs': hp.choice('epochs', [5, 10, 15]),
    'batch_size': hp.choice('batch_size', [8, 16, 32])
}

# Define the objective function to minimize
def objective(params):
    loss = lstm_model(params)
    return loss

# Perform Bayesian optimization
trials = Trials()
best_params = fmin(objective, space, algo=tpe.suggest, max_evals=10, trials=trials)
print("Best parameters:", best_params)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



  1/188 ━━━━━━━━━━━━━━━━━━━━ 25s 134ms/step - loss: 4.1132e-06
 67/188 ━━━━━━━━━━━━━━━━━━━━ 0s 771us/step - loss: 4.3158e-06 
136/188 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step - loss: 4.3950e-06
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 790us/step - loss: 4.4443e-06

  1/188 ━━━━━━━━━━━━━━━━━━━━ 24s 133ms/step - loss: 6.6039e-06
 63/188 ━━━━━━━━━━━━━━━━━━━━ 0s 810us/step - loss: 3.1390e-06 
123/188 ━━━━━━━━━━━━━━━━━━━━ 0s 825us/step - loss: 2.9557e-06
170/188 ━━━━━━━━━━━━━━━━━━━━ 0s 894us/step - loss: 2.8995e-06
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.9111e-06  

  1/188 ━━━━━━━━━━━━━━━━━━━━ 24s 133ms/step - loss: 2.0004e-05
 53/188 ━━━━━━━━━━━━━━━━━━━━ 0s 981us/step - loss: 2.1690e-05 
112/188 ━━━━━━━━━━━━━━━━━━━━ 0s 914us/step - loss: 2.4113e-05
172/188 ━━━━━━━━━━━━━━━━━━━━ 0s 888us/step - loss: 2.4499e-05
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 913us/step - loss: 2.4926e-05

  1/188 ━━━━━━━━━━━━━━━━━━━━ 26s 139ms/step - loss: 2.0206e-06
 57/188 ━━━━━━━━━━━━━━━━━━━━ 0s 904us/step - loss: 2.4518e-0

In [30]:

#create and fit the LSTM network
model = Sequential() 
model.add(LSTM(1, input_shape=(1, 1))) #4 LSTM blocks, 1 time step and 1 feature in each step
model.add(Dense(1)) # 1 fully connected (dense) layer with one neurone (or block)
optimizer = Adam(learning_rate= 0.022965568195725542)
model.compile(loss='mean_squared_error', optimizer=optimizer) #configures the model for training with the mean squared error loss function and the Adam optimizer
model.fit(train_data_x, train_data_y, epochs=5, batch_size=1, verbose=2) #trains the compiled model on the training data for 10 epochs using stochastic gradient descent with a batch size of 1, and it prints progress output for each epoch.

Epoch 1/5
5990/5990 - 6s - 1ms/step - loss: 0.0016
Epoch 2/5
5990/5990 - 5s - 897us/step - loss: 2.2747e-05
Epoch 3/5
5990/5990 - 10s - 2ms/step - loss: 2.2308e-05
Epoch 4/5
5990/5990 - 10s - 2ms/step - loss: 2.2923e-05
Epoch 5/5
5990/5990 - 10s - 2ms/step - loss: 2.2922e-05


In [31]:
#make predictions
test_predictions = model.predict(train_data_x)
test_predictions = test_predictions.flatten()
test_predictions = test_predictions[:len(test_data_x)]
test_data_xx = test_data_x.flatten()
print(test_predictions.shape)
print(test_data_xx.shape)


188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
(1498,)
(1498,)


In [32]:
mae = mean_absolute_error(test_data_xx, test_predictions)
print(mae)

0.14705691304340035


In [ ]:
#prédire les 100 prochaines valeurs
data = df['6'].values
data = data.reshape(data.shape[0], 1, 1)
predictions = model.predict(data)
predictions = predictions.flatten()
predictions = predictions[:100]
print(predictions.shape)
i = 0
with open("lstm_results.csv", 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['ID', 'Prediction'])
    for x in predictions:
        csv_writer.writerow([i, x])
        i += 1






 65/234 ━━━━━━━━━━━━━━━━━━━━ 0s 784us/step

234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 806us/step
(100,)
